In [1]:
from TIMIT.lightning_model_uncertainty_loss import LightningModel
import os
import pandas as pd
import numpy as np
import torch
import torchaudio
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from IPython import embed

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
model_checkpoint = 'checkpoints/epoch=12-step=6369-v3.ckpt'
model = LightningModel.load_from_checkpoint(model_checkpoint)
model.to('cuda')
model.eval()
print()

Using cache found in /root/.cache/torch/hub/s3prl_s3prl_master


[Downstream] can not import .sv_voxceleb1.expert: No module named 'utility'... Pass.
Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.

[hubconf] can not import .downstream.timit_phone.hubconf: No module named 'upstream'... Pass.
Using cache found in /root/.cache/torch/hub/s3prl_cache/3a990c945fbe378df95598eec534e91ba22a5d9eab0b2f88777a7a696d1344e9
for https://dl.fbaipublicfiles.com/fairseq/wav2vec/wav2vec_small.pt


/notebooks/s3prl_env/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `MeanAbsoluteError` was deprecated since v1.3.0 in favor of `torchmetrics.regression.mean_absolute_error.MeanAbsoluteError`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
/notebooks/s3prl_env/lib/python3.8/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `Accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.classification.accuracy.Accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)


Model Details: #Params = 164364294	#Trainable Params = 161212422



In [20]:
# model.model.transformer_encoder_M.layers[-5]

In [3]:
phnFiles = os.listdir('./TIMIT_Dataset/wav_data/phn')
testWavFiles = os.listdir('./TIMIT_Dataset/wav_data/TEST')
phnFiles.sort()
testWavFiles.sort()
print(len(phnFiles), len(testWavFiles))

6301 1680


In [4]:
phnFiles[:5]

['.ipynb_checkpoints',
 'FADG0_SA1.PHN',
 'FADG0_SA2.PHN',
 'FADG0_SI1279.PHN',
 'FADG0_SI1909.PHN']

In [5]:
testWavFiles[:5]

['FADG0_SA1.WAV',
 'FADG0_SA2.WAV',
 'FADG0_SI1279.WAV',
 'FADG0_SI1909.WAV',
 'FADG0_SI649.WAV']

In [6]:
testPhnWavFiles = []

for i,testWavFile in enumerate(testWavFiles):
    if((testWavFile[:-4]+'.PHN') in phnFiles):
        testPhnWavFiles.append(['./TIMIT_Dataset/wav_data/TEST/'+testWavFile, './TIMIT_Dataset/wav_data/phn/'+testWavFile[:-4]+'.PHN'])
len(testPhnWavFiles)

1680

In [7]:
testPhnWavFiles[:5]

[['./TIMIT_Dataset/wav_data/TEST/FADG0_SA1.WAV',
  './TIMIT_Dataset/wav_data/phn/FADG0_SA1.PHN'],
 ['./TIMIT_Dataset/wav_data/TEST/FADG0_SA2.WAV',
  './TIMIT_Dataset/wav_data/phn/FADG0_SA2.PHN'],
 ['./TIMIT_Dataset/wav_data/TEST/FADG0_SI1279.WAV',
  './TIMIT_Dataset/wav_data/phn/FADG0_SI1279.PHN'],
 ['./TIMIT_Dataset/wav_data/TEST/FADG0_SI1909.WAV',
  './TIMIT_Dataset/wav_data/phn/FADG0_SI1909.PHN'],
 ['./TIMIT_Dataset/wav_data/TEST/FADG0_SI649.WAV',
  './TIMIT_Dataset/wav_data/phn/FADG0_SI649.PHN']]

# Phone map

In [58]:
# https://catalog.ldc.upenn.edu/docs/LDC93S1/PHONCODE.TXT
symbolList = [                                      
                  'b',          
                  'd',          
                  'g',          
                  'p',          
                  't',          
                  'k',         
                  'dx',         
                  'q',          

                  'jh',         
                  'ch',         

                  's',          
                  'sh',         
                  'z',          
                  'zh',         
                  'f',          
                  'th',         
                  'v',           
                  'dh',         

                  'm',          
                  'n',         
                  'ng',         
                  'em',         
                  'en',         
                  'eng',        
                  'nx',         
                             
                  'l',          
                  'r',          
                  'w',          
                  'y',          
                  'hh',         
                  'hv',         
                  'el',         
                  
                  'iy',        
                  'ih',          
                  'eh',         
                  'ey',         
                  'ae',         
                  'aa',         
                  'aw',         
                  'ay',         
                  'ah',         
                  'ao',         
                  'oy',         
                  'ow',         
                  'uh',         
                  'uw',         
                  'ux',         
                  'er',         
                  'ax',        
                  'ix',        
                  'axr',        
                  'ax-h',      

                  'pau',     
                  'epi',     
                  'h#',     
                  '1',       
                  '2', 

                   'bcl','dcl','gcl','pcl','tck','kcl','tcl']    

In [59]:
len(symbolList)

64

In [60]:
from IPython import embed

In [65]:
def getSymbolInfo(phnFile, selfAttnMatrix):
    with open(phnFile) as f:
        lines = f.readlines()
    for line in lines:
        lo, hi, symbol = line.split(" ")
        lo = int(lo)
        hi = int(hi)
        if(symbol[-1] == '\n'):
            symbol = symbol[:-1]
        if(symbol == 'h#'):
            continue
#         print(lo, hi, symbol)
        if(symbol not in symbolList):
            embed()
        assert(symbol in symbolList)
        lo_ = lo//320
        hi_ = (hi+319)//320
        if(torch.isnan(selfAttnMatrix[lo_:hi_,lo_:hi_].mean())):
            embed()
        symbolCountMap[symbol] += selfAttnMatrix[lo_:hi_,lo_:hi_].mean()
        

# Making Plots

In [66]:
activation_M = {}
def get_activation_M(name):
    def hook(model, input, output):
        activation_M[name] = output
    return hook

activation_F = {}
def get_activation_F(name):
    def hook(model, input, output):
        activation_F[name] = output
    return hook

In [67]:
model.model.transformer_encoder_M.layers[5].self_attn.register_forward_hook(get_activation_M('attn_M'))
model.model.transformer_encoder_F.layers[5].self_attn.register_forward_hook(get_activation_F('attn_F'))

In [68]:
symbolCountMap = {x: 0 for x in symbolList}

for i,testPhnWavFile in enumerate(testPhnWavFiles):
    wav, _ = torchaudio.load(testPhnWavFile[0])
    wav_len = [wav.shape[1]]
    wav = wav.to('cuda')
    y_hat_h, y_hat_a, y_hat_g = model(wav, wav_len)
    y_hat_h = y_hat_h.to('cpu')
    y_hat_a = y_hat_a.to('cpu')
    y_hat_g = y_hat_g.to('cpu')
    
    phnFile = testPhnWavFile[1]
    
    if(y_hat_g>0.5):
        activation_F['attn_F']
        selfAttnMatrix = activation_F['attn_F'][1][0].detach().cpu()
        getSymbolInfo(phnFile, selfAttnMatrix)
    else:
        activation_M['attn_M']
        selfAttnMatrix = activation_M['attn_M'][1][0].detach().cpu()
        getSymbolInfo(phnFile, selfAttnMatrix)
        
#     if(i == 0):
#         break

In [69]:
symbolCountMap

{'b': tensor(6.0705),
 'd': tensor(8.1648),
 'g': tensor(5.1328),
 'p': tensor(6.0847),
 't': tensor(9.5571),
 'k': tensor(11.0887),
 'dx': tensor(5.9256),
 'q': tensor(7.9589),
 'jh': tensor(2.3525),
 'ch': tensor(1.6509),
 's': tensor(16.7183),
 'sh': tensor(4.8877),
 'z': tensor(7.9235),
 'zh': tensor(0.5042),
 'f': tensor(6.0633),
 'th': tensor(1.8119),
 'v': tensor(4.3701),
 'dh': tensor(6.8716),
 'm': tensor(10.0100),
 'n': tensor(15.6497),
 'ng': tensor(2.6711),
 'em': tensor(0.2944),
 'en': tensor(1.4855),
 'eng': tensor(0.0333),
 'nx': tensor(2.3813),
 'l': tensor(15.4387),
 'r': tensor(16.5119),
 'w': tensor(8.2950),
 'y': tensor(4.2079),
 'hh': tensor(2.4482),
 'hv': tensor(2.2969),
 'el': tensor(2.2406),
 'iy': tensor(17.6628),
 'ih': tensor(10.7860),
 'eh': tensor(9.5202),
 'ey': tensor(5.2584),
 'ae': tensor(9.5156),
 'aa': tensor(7.4007),
 'aw': tensor(1.5194),
 'ay': tensor(5.7596),
 'ah': tensor(5.6051),
 'ao': tensor(7.5102),
 'oy': tensor(1.9698),
 'ow': tensor(5.465

In [70]:
# import pickle
# with open('symbolCountMap.pkl', 'wb') as f:
#     pickle.dump(symbolCountMap, f)

In [71]:
# with open('symbolCountMap.pkl', 'rb') as f:
#     symbolCountMap = pickle.load(f)

In [72]:
# https://catalog.ldc.upenn.edu/docs/LDC93S1/PHONCODE.TXT
stopsSymbolList = [                                      
                  'b',          
                  'd',          
                  'g',          
                  'p',          
                  't',          
                  'k',         
                  'dx',         
                  'q',  
'bcl','dcl','gcl','pcl','tck','kcl']

affricatesSymbolList = [
                  'jh',         
                  'ch','dcl','tcl']     

fricativesSymbolList = [
                  's',          
                  'sh',         
                  'z',          
                  'zh',         
                  'f',          
                  'th',         
                  'v',           
                  'dh',]     
nasalsSymbolList = [
                  'm',          
                  'n',         
                  'ng',         
                  'em',         
                  'en',         
                  'eng',        
                  'nx',]       
semivowelsGlidesSymbolList = [                        
                  'l',          
                  'r',          
                  'w',          
                  'y',          
                  'hh',         
                  'hv',         
                  'el',]      
vowelsSymbolList = [            
                  'iy',        
                  'ih',          
                  'eh',         
                  'ey',         
                  'ae',         
                  'aa',         
                  'aw',         
                  'ay',         
                  'ah',         
                  'ao',         
                  'oy',         
                  'ow',         
                  'uh',         
                  'uw',         
                  'ux',         
                  'er',         
                  'ax',        
                  'ix',        
                  'axr',        
                  'ax-h',]    
othersSymbolList = [
                  'pau',     
                  'epi',     
                  'h#',     
                  '1',       
                  '2' ]    

In [73]:
stopsSymbolCount = 0
affricatesSymbolCount = 0
fricativesSymbolCount = 0
nasalsSymbolListCount = 0
semivowelsGlidesSymbolCount = 0
vowelsSymbolCount = 0
othersSymbolCount = 0

In [74]:
for (key,value) in symbolCountMap.items():
    if(key in stopsSymbolList):
        stopsSymbolCount += value
    if(key in affricatesSymbolList):
        affricatesSymbolCount += value
    if(key in fricativesSymbolList):
        fricativesSymbolCount += value
    if(key in nasalsSymbolList):
        nasalsSymbolListCount += value
    if(key in semivowelsGlidesSymbolList):
        semivowelsGlidesSymbolCount += value
    if(key in vowelsSymbolList):
        vowelsSymbolCount += value
    if(key in othersSymbolList):
        othersSymbolCount += value

In [75]:
categories = ['Stops', 'Affricates', 'Fricatives', 'Nasals', 'Semivowels and Glides', 'Vowels', 'Others']
values = np.array([stopsSymbolCount, affricatesSymbolCount, fricativesSymbolCount, nasalsSymbolListCount, semivowelsGlidesSymbolCount, vowelsSymbolCount, othersSymbolCount])
values = values/max(values)
print(values)

[0.7396303  0.21688189 0.36126614 0.2390644  0.37808675 1.
 0.04179452]


In [76]:
plt.barh(categories, values)
plt.xlabel("Relative importance")
plt.ylabel("Types of phone")
plt.title("")
plt.savefig('phoneGraph.png', dpi=600, bbox_inches='tight')
plt.show()

In [15]:
max(values)

122898

In [156]:
model.uncertainty_loss.log_var_height

Parameter containing:
tensor(-0.0056, device='cuda:0', requires_grad=True)

In [157]:
model.uncertainty_loss.log_var_age

Parameter containing:
tensor(-0.0035, device='cuda:0', requires_grad=True)

In [158]:
model.uncertainty_loss.log_var_gender

Parameter containing:
tensor(-0.0065, device='cuda:0', requires_grad=True)